# Lab 2. Solar Panel Instructions

## Introduction

In this notebook we show you how to create your second sub-agent on Amazon Bedrock Agents.

This agent contains instructions on how to install and to do maintenance on Solar Panels, where customers can ask the agent to return these information from an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/).

To equip foundation models (FMs) with up-to-date and proprietary information, organizations use Retrieval Augmented Generation (RAG), a technique that fetches data from company data sources and enriches the prompt to provide more relevant and accurate responses. 

Amazon Bedrock Knowledge Bases is Bedrock's fully managed capability that helps you implement the entire RAG workflow from ingestion to retrieval and prompt augmentation without having to build custom integrations to data sources and manage data flows.

In the context of our agent, if the answer to a question is not present on the Knowledge Base, customers can ask agent to create a support ticket, to get a human in the loop to help with their questions.

The following represents the piece of architecture that will be built on this module.

![Architecture](img/solar_panel_agent.png)

## Setup

Firstly, you are going to install boto3 dependencies from pip. Make sure to have version superior of **1.35.45**

In [ ]:
# Install Dependencies from local package
!pip install ../boto3/controlplane/botocore-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/boto3-1.35.55-py3-none-any.whl \
    ../boto3/controlplane/awscli-1.35.21-py3-none-any.whl --force-reinstall --no-cache

One component of a Knowledge Base is a Vector Database. Vector database is a kind of database that is optimized to store and search with vectors, making RAG architectures possible and faster. 

Amazon Bedrock Knowledge Bases is powered by one vector database. 

For this lab we are going to create one collection using [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/). If you want to know more, [this link](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) contains other supported vector engines for Amazon Bedrock Knowledge Bases.

Once you select your vector database, you need to populate it with documents indexed from a data storage. Amazon Bedrock Kwnoledge Bases allow you to use different [data connectors](https://docs.aws.amazon.com/bedrock/latest/userguide/data-source-connectors.html) to populate your vector database. For this lab we will store our data in an Amazon S3 bucket and connect it with our knowledge base.

Now, let's install opensearch-py to interact with Opensearch engine.

In [ ]:
!pip install opensearch-py retrying --force-reinstall --no-cache

Restart kernel for packages to take effect

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

Check if your boto3 version is superior than **1.35.45**

In [ ]:
!pip freeze | grep boto3

## Creating Agent

On this section we declare global variables that will be act as helpers during entire notebook and we will start to create out second agent.

In [ ]:
import boto3
import os
import sys

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"
s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'anthropic.claude-3-5-sonnet-20240620-v1:0'
]

In [ ]:
solar_agent_name = f"solar-p-{agent_suffix}"

solar_lambda_name = f"fn-solar-p-{agent_suffix}"

solar_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{solar_agent_name}'

dynamodb_table = f"{solar_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "ticket_id"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{solar_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing solar panel instructions for installation and maintenance"
bucket_name = f'{solar_agent_name}-{suffix}'

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the file can be recognized and invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(3, "../../../..")

from src.utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from src.utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and Load Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

In the next steps we will generate the data used to populate the knowledge base. It will be composed of instructions on how to handle a solar panel

**This creation process takes around 5min.**

In [ ]:
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

### Create Synthetic Data to Load on S3

Instead of get data elsewhere, you're going to generate data, using a LLM on Amazon Bedrock.
This fake data that will be generated, will be uploaded into a S3 bucket and then added into an Amazon Bedrock Knowledge Base.

In [ ]:
path = "kb_documents"
# Check whether the specified path exists or not
isExist = os.path.exists(path)
if not isExist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Creating helper methods to invoke LLM on Bedrock and to write a local file using Python

In [ ]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            { "text": prompt } 
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 500,
            "temperature": 0
        },
    )

    return response['output']['message']

def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close

Generating one file with instructions on how to install a solar panel.

In [ ]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step on how to install a solar panel at home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-instructions.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])

### Generating data prompt
Generating another file with instructions on how to do maintenance on a solar panel.

In [ ]:
text_generation_energy_instructions = '''
    You will be act as an expert on clean energy.
    You will generate a step-by-step instructions on how to do maintenance in a solar panel installed at a regular home.
    Just answer in a numbered list.
'''

solar_energy_file_name = 'solar-panel-maintenance.txt'

response_message = invoke_bedrock_generate_energy_files(text_generation_energy_instructions)

write_file('{}/{}'.format(path, solar_energy_file_name),response_message['content'][0]['text'])


### Uploading data to S3
Uploading generated files into S3 Bucket.

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### Synchronizing Knowledge Base
Now that the data is available in the s3 bucket, let's synchronize it to our knowledge base

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## Creating Agent

Create the Solar Panel agent that will have a Knowledge Base and also a Lambda action group to handle exception workflow (when information is not found on KB).

For this agent we will use the following instructions:
```
You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Providing maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner
5. NEVER invent information not available in your knowledge base

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user
```
We will also connect a knowledge base with the information about solar panels

```
Access the knowledge base when customers ask about to install and maintain solar panels
```
And we will make the following tool available to the agent:
- `open_ticket`: to open new support tickets
- `get_ticket_status`: to get the current status of an existing ticket

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about to install and maintain solar panels'
}

In [ ]:
agent_instruction = """You are a Solar Energy Assistant that helps customers with solar panel installation and maintenance guidance.

Your capabilities include:
1. Providing installation instructions
2. Offering maintenance procedures
3. Troubleshooting common issues
4. Creating support tickets for specialist assistance

Core behaviors:
1. Always use available information before asking customers for additional details
2. Maintain a professional yet approachable tone
3. Provide clear, direct answers
4. Present technical information in an easy-to-understand manner

Support ticket protocol:
- Only generate tickets for specialist-level issues
- Respond exclusively with case ID when creating tickets
- Decline providing specialist advice beyond your scope

Response style:
- Be helpful and solution-oriented
- Use clear, practical language
- Focus on actionable guidance
- Maintain natural conversation flow
- Be concise yet informative
- Do not add extra information not required by the user"""

agent_description = """You are a solar energy helper bot. 
    You can retrieve information on how to install and do maintenance on solar panels"""

solar_agent = agents.create_agent(
    solar_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn]
)
solar_agent

In [ ]:
agents.associate_kb_with_agent(
    solar_agent[0], kb_config['kb_instruction'], kb_config['kb_id']
)

### Creating Action Group

On this session, we're going create an action group to handle support tickets and associate it with our agent. To do so, we will first create a Lambda function code to fulfill the execution of the agent's actions Next we will define the actions available actions that an agent can take using function details. Similar to the previous agent, you can also define the actions available using OpenAPI Schema.

#### Creating Lambda function
First let's create the lambda function

In [ ]:
%%writefile solar_energy.py
import os
import json
import uuid
import boto3

from boto3.dynamodb.conditions import Key, Attr

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(table_name: str, 
                   pk_field: str,
                   pk_value: str,
                   sk_field: str=None, 
                   sk_value: str=None):
    try:
        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_value:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).begins_with(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        query_data = table.query(
            KeyConditionExpression=key_expression
        )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def open_ticket(customer_id, msg):
    ticket_id = str(uuid.uuid1())
    item = {
        'ticket_id': ticket_id,
        'customer_id': customer_id,
        'description': msg,
        'status': 'created'
    }
    resp = put_dynamodb(dynamodb_table, item)
    print(resp)
    return "Thanks for contact customer {}! Your support case was generated with ID: {}".format(
        customer_id, ticket_id
    )

def get_ticket_status(customer_id,
                      ticket_id: str=None):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk,
                         customer_id,
                         dynamodb_sk,
                         ticket_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'open_ticket':
        msg = get_named_parameter(event, "msg")
        result = open_ticket(customer_id, msg)
    elif function == 'get_ticket_status':
        ticket_id = get_named_parameter(event, "ticket_id")
        result = get_ticket_status(customer_id, ticket_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

### Defining available actions
Now it's time to define the actions that can be taken by the agent

In [ ]:
functions_def =[
    {
        "name": "open_ticket",
        "description": """Create a ticket to get help with information related with solar panel or clean energy""",
        "parameters": {
                        "customer_id": {
                            "description": "Unique customer identifier",
                            "required": True,
                            "type": "string"
                        },
                        "msg": {
                            "description": "The reason why customer is opening a ticket",
                            "required": True,
                            "type": "string"
                        }
                    }
    },
    {
        "name": "get_ticket_status",
        "description": """get the status of an existing ticket""",
        "parameters": {
                        "customer_id": {
                            "description": "Unique customer identifier",
                            "required": True,
                            "type": "string"
                        },
                        "ticket_id": {
                            "description": "Unique ticket identifier",
                            "required": False,
                            "type": "string"
                        }
                    }
    }
]

#### Associating action group to agent
Finally, we can associate a new action group with our previously created agent

In [ ]:
resp = agents.add_action_group_with_lambda(
    agent_name=solar_agent_name,
    lambda_function_name=solar_lambda_name,
    source_code_file="solar_energy.py",
    agent_functions=functions_def,
    agent_action_group_name="solar_energy_actions",
    agent_action_group_description="Function to open an energy ticket for a user or get status from an opened ticket",
    dynamo_args=dynamoDB_args
)

## Testing Agent

Now, let's run some tests on the agent we just created to make sure it's working. To do so we will use our test alias: `TSTALIASID` which allows you to invoke a draft version of your agent

### Testing maintainance question
First let's ask a question related to maintaining an existing solar panel

In [ ]:
%%time
print(
    agents.invoke(
        "how can I check if my solar panel eletrical consumption is compliant with energy rules?", 
        solar_agent[0]
    )
)

### Testing installation question
Next let's ask a question related to installing a new solar panel

In [ ]:
%%time
print(
    agents.invoke(
        "how can I install a solar pannel at home?", 
        solar_agent[0]
    )
)

### Testing personalized support
Let's now create a support ticket. To do so we will pass our customer id to trigger the support ticket creation process

In [ ]:
%%time
print(
    agents.invoke(
        "Can I get support to install my solar panel? My customer id is 1", 
        solar_agent[0]
    )
)

### Testing getting support ticket details
Now let's get the details for our support ticket. For that you will need to provide the ticket id just generated in the previous query to `create_ticket`

In [ ]:
created_ticket = ""

In [ ]:
%%time
print(
    agents.invoke(
        "Can I get details on my ticket? My customer id is 1 and my ticket id is {}".format(created_ticket), 
        solar_agent[0]
    )
)

### Testing get open support tickets
Let's also check if our agent can get the details for more than one open case. To do so we will first create a new support case

In [ ]:
%%time
print(
    agents.invoke(
        "Can I get support to review my solar panel consumption? My customer id is 1", 
        solar_agent[0]
    )
)

Next we will request the overview of all open tickets

In [ ]:
%%time
print(
    agents.invoke(
        "Can I get all tickets that I have? My customer id is 1", 
        solar_agent[0]
    )
)

## Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [ ]:
solar_agent_alias_id, solar_agent_alias_arn = agents.create_agent_alias(
    solar_agent[0], 'v1'
)
solar_agent_id = solar_agent[0]

Store environment variables to be used on next notebooks.

In [ ]:
solar_agent_arn = agents.get_agent_arn_by_name(solar_agent_name)
solar_kb = knowledge_base_name
solar_dynamodb = dynamodb_table

%store solar_agent_arn
%store solar_agent_alias_arn
%store solar_agent_alias_id
%store solar_lambda_name
%store solar_agent_name
%store solar_agent_id
%store solar_kb
%store solar_dynamodb

In [ ]:
solar_agent_arn, solar_agent_alias_arn, solar_agent_alias_id

## Next Steps
Congratulations! We've now created a solar panel agent. Next we will create our peak loader manager agent